# Random baseline

In [1]:
from datasets import load_from_disk
from src.utils.config_management import CONFIG

## Data loading

In [2]:
hf_dataset = load_from_disk(CONFIG['paths']['data']['dalip_hf_dataset'])

In [3]:
hf_dataset

DatasetDict({
    train: Dataset({
        features: ['answer_id', 'question_id', 'answer_creation_date', 'answer_score', 'answer_normalized_score', 'answer_log_normalized_score', 'answer_body', 'answer_last_edit_date', 'answer_last_activity_date', 'answer_comment_count', 'answer_community_owned_date', 'question_creation_date', 'question_score', 'question_view_count', 'question_body', 'question_last_edit_date', 'question_last_activity_date', 'question_title', 'question_tags', 'question_answer_count', 'question_comment_count', 'question_favorite_count', 'question_closed_date', 'question_community_owned_date', 'answer_accepted'],
        num_rows: 42700
    })
    test: Dataset({
        features: ['answer_id', 'question_id', 'answer_creation_date', 'answer_score', 'answer_normalized_score', 'answer_log_normalized_score', 'answer_body', 'answer_last_edit_date', 'answer_last_activity_date', 'answer_comment_count', 'answer_community_owned_date', 'question_creation_date', 'question_score', 

## Create predictions

In [4]:
import numpy as np

In [5]:
TARGET_COL = 'answer_normalized_score'

In [6]:
y_train = hf_dataset['train'][TARGET_COL]
y_test = hf_dataset['test'][TARGET_COL]

In [7]:
np.random.seed(42)
predictions = np.random.choice(y_train, len(y_test), replace=True)

## Evaluation

In [8]:
import pandas as pd
import wandb
from collections import defaultdict
from src.evaluation import RankingEvaluator

In [9]:
wandb_config = defaultdict(dict)

wandb_config['regression_model']['regressor_type'] = 'random sampling'

wandb_config

defaultdict(dict, {'regression_model': {'regressor_type': 'random sampling'}})

In [25]:
run = wandb.init(
    project='dalip-stackoverflow-answer-ranking',
    config=wandb_config,
    tags=['random', 'baseline']
)

### Compute metrics

In [10]:
test_question_ids = hf_dataset['test']['question_id']

In [11]:
evaluator = RankingEvaluator(ndcg_k=list(range(1, 11)), 
                             ndcg_gain_func='linear', ndcg_discount_func='logarithmic')

In [12]:
metrics = evaluator(y_test, predictions, test_question_ids)
if TARGET_COL == 'answer_log_normalized_score':
    metrics.pop('mae')
metrics

{'ndcg@1_g.linear_d.logarithmic': 0.8461911645058998,
 'ndcg@2_g.linear_d.logarithmic': 0.8855314584320246,
 'ndcg@3_g.linear_d.logarithmic': 0.9129489387520202,
 'ndcg@4_g.linear_d.logarithmic': 0.9348100223472279,
 'ndcg@5_g.linear_d.logarithmic': 0.9453848970022316,
 'ndcg@6_g.linear_d.logarithmic': 0.9508231427568997,
 'ndcg@7_g.linear_d.logarithmic': 0.9535565001594575,
 'ndcg@8_g.linear_d.logarithmic': 0.9554812919621696,
 'ndcg@9_g.linear_d.logarithmic': 0.9566379414059627,
 'ndcg@10_g.linear_d.logarithmic': 0.9573032527257985,
 'mae': 3.669601438985137,
 'hit_rate@1': 0.25861182519280207}

In [30]:
wandb.log(metrics)

### Log predictions

In [31]:
predictions_df = pd.DataFrame()
predictions_df['answer_id'] = hf_dataset['test']['answer_id']
predictions_df['answer_normalized_score'] = hf_dataset['test']['answer_normalized_score']
predictions_df['predicted_score'] = predictions

In [32]:
wandb.log({'predictions_table': wandb.Table(dataframe=predictions_df)})

In [33]:
run.finish()

hit_rate@1,▁
mae,▁
ndcg@10_g.exponential_d.logarithmic,▁
ndcg@1_g.exponential_d.logarithmic,▁
ndcg@2_g.exponential_d.logarithmic,▁
ndcg@3_g.exponential_d.logarithmic,▁
ndcg@4_g.exponential_d.logarithmic,▁
ndcg@5_g.exponential_d.logarithmic,▁
ndcg@6_g.exponential_d.logarithmic,▁
ndcg@7_g.exponential_d.logarithmic,▁
ndcg@8_g.exponential_d.logarithmic,▁
